In [1]:
# 이거 실행(설치)하고 런타임 재시작 하면 한글 출력 가능
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (9,226 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 한글 폰트 출력
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic') 

###데이터준비

In [3]:
path = '/content/drive/MyDrive/데이터분석/2차_프로젝트_lmemdata/lmemdata/'

In [4]:
purchase_tr_path = '구매상품TR.txt'
merchant_cl_path = '상품분류.txt'
compet_use_path = '경쟁사이용.txt'
cust_demo_path = '고객DEMO.txt'
channel_use_path = '채널이용.txt'
membership_path = '멤버십여부.txt'

In [5]:
import pandas as pd

In [6]:
mem = pd.read_csv(path+membership_path, encoding='cp949')   # 멤버쉽이 있는 회원은 기입됨
cha = pd.read_csv(path+channel_use_path, encoding='cp949') # 멤버들이 어떤 채널로 사용하는지
cust = pd.read_csv(path+cust_demo_path, encoding='cp949') # 멤버들의 기본 신상 정보
compet = pd.read_csv(path+compet_use_path, encoding='cp949') # 멤버들이 경쟁 회사를 사용하는지
pro = pd.read_csv(path+merchant_cl_path, encoding='cp949') # 상품 정보

In [7]:
# 대용량 데이터 불러오기 (transaction은 용량이 겁나 큼)
purch = pd.read_csv(path+purchase_tr_path, encoding='cp949', chunksize = 1000000) #100만개 단위로 끊기, 실제 transaction 데이터
chunk = list(purch)

print(len(chunk)) # 몇개 chunk인지 출력

purch = pd.concat(chunk)  # 데이터 넣기
purch.shape

29


(28593030, 10)

In [8]:
purch['구매일자'] = pd.to_datetime(purch['구매일자'],format='%Y%m%d')

In [57]:
purch.head()

,제휴사,영수증번호,대분류코드,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액
0,B,8664000,15,1504,B150401,17218,44,2014-02-22,20,2420
1,B,8664000,16,1601,B160101,17218,44,2014-02-22,20,1070
2,B,8664000,16,1602,B160201,17218,44,2014-02-22,20,8060
3,B,8664000,18,1803,B180301,17218,44,2014-02-22,20,6000
4,B,8664001,5,509,B050901,17674,44,2014-02-22,22,1120


In [74]:
purch[purch['고객번호'] == 10000]['제휴사'].value_counts()

C    1781
A       7
B       3
Name: 제휴사, dtype: int64

In [63]:
purch_sum = purch.groupby(['고객번호'], as_index=False)['구매금액'].sum()
purch_sum.describe()

,고객번호,구매금액
count,19383.00000,1.938300e+04
mean,9692.00000,3.492850e+07
std,5595.53447,6.435374e+07
min,1.00000,1.549000e+04
25%,4846.50000,1.093242e+07
50%,9692.00000,1.717624e+07
75%,14537.50000,3.935635e+07
max,19383.00000,3.571832e+09


###성별

In [17]:
f_cust = cust[cust['성별'] == 'F']['고객번호'].values

In [37]:
purch_copy = purch.copy()

In [35]:
m_cust = []
for i in range(len(purch)):
  if purch['고객번호'][i] not in f_cust:
    m_cust.append(i)

f_purch = purch_copy.drop(m_cust, axis=0, inplace=False)

In [44]:
f_purch['제휴사'].value_counts()

B    10907028
C     7584482
A     4800576
D       80353
Name: 제휴사, dtype: int64

In [53]:
f_purch.head()

,제휴사,영수증번호,대분류코드,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액
0,B,8664000,15,1504,B150401,17218,44,2014-02-22,20,2420
1,B,8664000,16,1601,B160101,17218,44,2014-02-22,20,1070
2,B,8664000,16,1602,B160201,17218,44,2014-02-22,20,8060
3,B,8664000,18,1803,B180301,17218,44,2014-02-22,20,6000
4,B,8664001,5,509,B050901,17674,44,2014-02-22,22,1120


In [56]:
f_purch.groupby(['고객번호'])['영수증번호'].count()

고객번호
4         930
6        1593
7         781
9         740
10       3408
         ... 
19379     584
19380     440
19381     805
19382     906
19383     260
Name: 영수증번호, Length: 15898, dtype: int64

###온라인이용 고객 데이터 분석


In [ ]:
cha.groupby(['제휴사'])[['이용횟수']].sum()

,이용횟수
제휴사,
A_MOBILE/APP,13256
B_MOBILE/APP,93067
B_ONLINEMALL,8978
C_MOBILE/APP,56
C_ONLINEMALL,3770
D_MOBILE/APP,609


###C 제휴사 점포분석

####준비단계

In [ ]:
purch.head()

,제휴사,영수증번호,대분류코드,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액
0,B,8664000,15,1504,B150401,17218,44,2014-02-22,20,2420
1,B,8664000,16,1601,B160101,17218,44,2014-02-22,20,1070
2,B,8664000,16,1602,B160201,17218,44,2014-02-22,20,8060
3,B,8664000,18,1803,B180301,17218,44,2014-02-22,20,6000
4,B,8664001,5,509,B050901,17674,44,2014-02-22,22,1120


In [ ]:
purch.shape

(28593030, 10)

In [ ]:
# 제휴사별 점포수
purch.groupby(['제휴사'])[['점포코드']].count()

,점포코드
제휴사,
A,5770318
B,13338074
C,9379236
D,105402


In [ ]:
# 제휴사, 점포별 구매금액 오름차순
purch.groupby(['제휴사','점포코드'])[['구매금액']].sum().sort_values('구매금액')[:30]

구매금액
제휴사 점포코드        
C   588     2990
    439     9970
    445    12470
    408    23240
    326    31050
    523    41240
    564    43820
    399    44700
    414    56640
    554    57940
    468    66574
    589    71250
    447    72800
    591    76070
    582    78501
    432    99232
    400   116597
    426   119030
    452   123894
    440   130432
    437   137590
    431   137830
    442   155030
D   52    165980
C   446   185490
D   53    194760
C   545   214988
    586   221560
    392   229033
    106   236339

In [ ]:
# purch_c = purch[purch['제휴사'] == 'C']

# C 제휴사 점포별 매출 오름차순
# purch_c.groupby('점포코드')[['구매금액']].sum().sort_values('구매금액')

# C 제휴사 점포 평균매출
# np.mean(purch_c.groupby('점포코드')['구매금액'].sum()) # C제휴사 점포 평균 구매금액 84419850원

# 점포별 매출
purch_c.groupby('점포코드')[['구매금액']].sum()
# c_grouped = purch_c.groupby('점포코드')[['구매금액']].sum()

,구매금액
점포코드,
1,762381245
2,93356905
3,145647075
4,316993548
5,115056794
...,...
589,71250
590,438290
591,76070


In [ ]:
c_grouped[c_grouped['구매금액'] < 84419850].sort_values('구매금액')[:30] # 평균 이하 매출 점포 401개. 그중 top30

,구매금액
점포코드,
588,2990
439,9970
445,12470
408,23240
326,31050
523,41240
564,43820
399,44700
414,56640


####매출 낮은 점포 판매분석

In [ ]:
# 점포별, 중분류코드별 구매금액을 오름차순으로 정렬 후 가장 낮은 30개 점포코드와 중분류코드 추출
purch_c.groupby(['점포코드','중분류코드'])[['구매금액']].sum().sort_values('구매금액')[:30]

In [ ]:
# 상품 분석
# pro_c = pro[pro['제휴사'] == 'C']
# 구매 분석
# purch_c[purch_c['점포코드'] == 588] # 18775 고객이 15/11/11 봉지라면 구매

# purch_c[purch_c['점포코드'] == 439] # 2255 고객이 15/1/1 계란, 냉장떡 구매

# purch_c[purch_c['점포코드'] == 445] # 19097 고객이 14/1/10 귤, 돼지고기앞다리, 14/1/16 귤, 두부, 숙주나물 구매

# purch_c[purch_c['점포코드'] == 408] # 14488 고객이 15/11/17 기름종이, 14/12/20 담배, 봉지라면, 컵라면 구매
                                    # 18798 고객이 14/3/3 우유, 비타민(200원), 캔디, 14/5/7 비타민(3980원), 
                                    # 14/11/19 과일음료, 소주, 맥주, 공병, 비타민(1000원) 구매

In [ ]:
purch_c[purch_c['점포코드'] == 445]

,제휴사,영수증번호,대분류코드,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액
14633717,C,10379145,1,101,C010104,19097,445,2014-01-10,17,2500
14633718,C,10379145,5,502,C050205,19097,445,2014-01-10,17,3500
14633719,C,10379146,1,101,C010104,19097,445,2014-01-16,19,3000
14633720,C,10379146,3,304,C030401,19097,445,2014-01-16,19,2000
14633721,C,10379146,3,304,C030405,19097,445,2014-01-16,19,1470


18775 고객은 데이터상 유일한 588 점포 사용객이다. 다른 제휴사 상품도 구매한 데이터가 있다. 

따라서 588 점포의 낮은 매출량과 연관 짓기에는 고객의 특성이 아닐 가능성이 높다. 그러면 588, 439, 445, 408 점포의 문제는 무엇일까?

In [ ]:
# 18775 고객 구매분석
purch[purch['고객번호'] == 18775].groupby(['제휴사','점포코드'])['구매금액'].sum() # 18775 고객이 B76, A11, C588 점포 이용, 
                                                                          # 각각 6942648, 319500, 2990원어치 구매


####매출 낮은 점포 위치분석

In [ ]:
purch_c

,제휴사,영수증번호,대분류코드,중분류코드,소분류코드,고객번호,점포코드,구매일자,구매시간,구매금액
1771,C,10805593,3,305,C030501,5836,299,2014-05-15,18,1490
1772,C,10805593,6,604,C060401,5836,299,2014-05-15,18,1290
1773,C,10805594,16,1602,C160201,5836,299,2014-05-15,18,11900
1774,C,10805595,5,502,C050209,5836,299,2014-05-15,18,9836
1775,C,10805596,5,502,C050211,11348,299,2014-05-15,18,22767
...,...,...,...,...,...,...,...,...,...,...
24982094,C,9673762,13,1304,C130401,17189,352,2014-08-03,17,26200
24982095,C,9673762,16,1604,C160406,17189,352,2014-08-03,17,31000
24982096,C,9673763,3,305,C030507,17139,352,2014-08-04,18,3000
24982097,C,9673764,1,101,C010116,3004,352,2014-08-04,18,4990


In [ ]:
# cust[cust['고객번호'] == 18775]

# 해당점포 주요이용고객 top50 뽑아내기
c_top50 = purch_c.groupby(['점포코드','고객번호'], 
                          as_index=False)[['영수증번호']].count().sort_values('영수증번호', ascending=False)[:50]

# 주요이용고객 거주지역

In [ ]:
c_top50

,점포코드,고객번호,영수증번호
27628,410,12871,10182
22788,312,8024,6663
20624,283,16747,6291
22723,312,4018,6070
1328,13,14858,5854
20713,285,16840,5530
8601,95,9724,5527
3729,40,6030,5476
24491,331,13148,5337
24024,325,144,5311


#### 매출 낮은 중분류

In [ ]:
# pro_c.중분류명.unique()
purch_c.groupby(['중분류코드'])[['구매금액']].sum().sort_values('구매금액')[:30]

# pro_c[pro_c['중분류코드'] == 1707] # 가정잡화 점포 전체적으로 매출 낮음
# pro_c[pro_c['중분류코드'] == 1206] # 기타음주류 공병/공박스 점포 전체적으로 매출 낮음
# pro_c[pro_c['중분류코드'] == 203] # 미숫가루, 선식류가 중분류 중 가장 매출 낮음
# pro_c[pro_c['중분류코드'] == 1204] # 양주가 중분류 중 세 번째로 매출 낮음
# pro_c[pro_c['중분류코드'] == 1203] # 와인이 네 번째


,구매금액
중분류코드,
203,5648930
1206,19427020
1204,25827070
1203,35775280
1003,41587820
601,52031027
1507,60387113
104,75755050
1105,78481985


###A 제휴사

####매출 낮은 중분류

In [ ]:
# purch_a = purch[purch['제휴사'] == 'A']
# pro_a = pro[pro['제휴사'] == 'A']

purch_a.groupby(['중분류코드'])[['구매금액']].sum().sort_values('구매금액')[:30]
# pro_a[pro_a['중분류코드'] == 407] # 블라우스 매출 2432100
# pro_a[pro_a['중분류코드'] == 907] # 웨딩 매출 4758100

,구매금액
중분류코드,
407,2432100
907,4758100
111,62550280
504,73985250
408,241133900
704,289119650
705,345356700
902,634085060
107,1599773620


###B 제휴사

####매출 낮은 중분류

In [ ]:
# purch_b = purch[purch['제휴사'] == 'B']
# pro_b = pro[pro['제휴사'] == 'B']

purch_b.groupby(['중분류코드'])[['구매금액']].sum().sort_values('구매금액')[:30]

# pro_b[pro_b['중분류코드'] == 103] # 온라인채소 매출 5900
# pro_b[pro_b['중분류코드'] == 4805] # 온라인돼지고기 매출 6500
# pro_b[pro_b['중분류코드'] == 7705] # 온라인규격/간편조리생선 매출 7500
# pro_b[pro_b['중분류코드'] == 4211] # 전자악기 매출 7600
# pro_b[pro_b['중분류코드'] == 9103] # 온라인뿌리채소 매출 9900

,구매금액
중분류코드,
103,5900
4805,6500
7705,7500
4211,7600
9103,9900
2813,12400
9204,14900
9004,20300
4201,23000


In [ ]:
# 정규표현식으로 온라인판매상품 찾기
li_ct = pro_b['중분류명'].unique().astype('str')
li_ct

import re
p = re.compile('온라인+')
li = []
for ct in li_ct:
  if p.match(ct):
    li.append(ct)
li

['온라인버섯/열매채소',
 '온라인잎/샐러드채소',
 '온라인돼지고기',
 '온라인양말',
 '온라인잡곡선식',
 '온라인규격/간편조리생선',
 '온라인해물연체',
 '온라인화장품',
 '온라인양념/뿌리채소',
 '온라인저장과일']

In [ ]:
# 온라인상품 중분류코드 찾기
code = []
for i in range(len(li)):
  code.append(pro_b[pro_b['중분류명'] == li[i]]['중분류코드'].values[0])

code

[103, 511, 4805, 6807, 7203, 7705, 7907, 9004, 9103, 9204]

In [ ]:
for i in range(len(code)):
  print(purch_b[purch_b['중분류코드'] == code[i]].index)

Int64Index([21652248], dtype='int64')
Int64Index([21652177, 21652261], dtype='int64')
Int64Index([26570401], dtype='int64')
Int64Index([10086011, 20291718, 26073136], dtype='int64')
Int64Index([21652246, 26570377], dtype='int64')
Int64Index([10317608], dtype='int64')
Int64Index([21740624], dtype='int64')
Int64Index([21652170, 26530735], dtype='int64')
Int64Index([21652178], dtype='int64')
Int64Index([26570393], dtype='int64')


기존 점포 분석해서 인사이트 얻기
- 매출 낮은 점포의 원인 분석 (재고관리 등 개선 필요)
- 상권변화로 고객이 줄어든 경우 -> 신규점포 대안

고객별 구매금액

In [ ]:
# 고객별 구매금액
user_df = pd.DataFrame(purch.groupby(['고객번호','구매일자']).구매금액.sum())
user_df = user_df.swaplevel(0, 1, axis = 0)
user_df = user_df.unstack().resample('M').sum()   # 개별 고객의 월별 구매금액 합계
user_df = user_df['구매금액']   # column: 고객번호, row: 구매 월

# # 1번 10번까지 고객의 매달 구매금액 합계 그래프
# plt.figure(figsize = (15,10))
# for i in range (10) :
#   plt.plot(user_df[i+1]);

In [ ]:
user_df

고객번호,1,2,3,4,5,6,7,8,9,10,...,19374,19375,19376,19377,19378,19379,19380,19381,19382,19383
구매일자,,,,,,,,,,,,,,,,,,,,,
2014-01-31,2307260.0,4542290.0,128369.0,413200.0,1175800.0,803340.0,6924763.0,2707312.0,2418110.0,1047403.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-02-28,3495770.0,3846010.0,96393.0,452663.0,639880.0,581668.0,2988510.0,2092350.0,786550.0,874437.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-03-31,3338560.0,5735170.0,78020.0,443780.0,1329650.0,780998.0,722600.0,1631450.0,11158930.0,623233.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-04-30,3969945.0,2766880.0,115200.0,1328170.0,825790.0,1232800.0,792140.0,1348690.0,8675910.0,699681.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-05-31,2106840.0,4400530.0,166150.0,1657533.0,559180.0,1149520.0,924442.0,1812910.0,7636200.0,761279.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-06-30,4706980.0,3032350.0,267083.0,746760.0,929850.0,732964.0,1125740.0,2102670.0,7591720.0,1124893.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-07-31,3105200.0,4423460.0,233918.0,1071390.0,882620.0,1068331.0,1562350.0,1071570.0,7823540.0,809674.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-08-31,4939610.0,3920190.0,200320.0,586250.0,795880.0,1381112.0,2435839.0,1931400.0,4216730.0,749018.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-30,2736740.0,6497530.0,213950.0,776770.0,148790.0,713830.0,713380.0,3996530.0,4870810.0,694242.0,...,0.0,0.0,0.0,0.0,0.0,130300.0,0.0,0.0,0.0,0.0


In [ ]:
# 2015년에 구매금액이 2014년 비교해서 감소한 고객 비율
user_by_year = user_df.resample('Y').sum()
user_by_year.loc['2014/2015 매출비율'] = user_by_year.loc['2014-12-31']/user_by_year.loc['2015-12-31']
(user_by_year.loc['2014/2015 매출비율'] > 1).sum() / 19383

0.44533869885982563

In [ ]:
user_df.resample('Q').sum()

고객번호,1,2,3,4,5,6,7,8,9,10,...,19374,19375,19376,19377,19378,19379,19380,19381,19382,19383
구매일자,,,,,,,,,,,,,,,,,,,,,
2014-03-31,9141590.0,14123470.0,302782.0,1309643.0,3145330.0,2166006.0,10635873.0,6431112.0,14363590.0,2545073.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-06-30,10783765.0,10199760.0,548433.0,3732463.0,2314820.0,3115284.0,2842322.0,5264270.0,23903830.0,2585853.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-30,10781550.0,14841180.0,648188.0,2434410.0,1827290.0,3163273.0,4711569.0,6999500.0,16911080.0,2252934.0,...,0.0,0.0,0.0,0.0,0.0,130300.0,0.0,0.0,0.0,0.0
2014-12-31,23334762.0,7080920.0,394366.0,1823100.0,398440.0,2950816.0,16221723.0,11151680.0,22951170.0,1971990.0,...,817925.0,1159694.0,273297.0,217360.0,474760.0,1060320.0,522700.0,0.0,0.0,0.0
2015-03-31,9302985.0,18481480.0,279310.0,1737292.0,2124430.0,3619234.0,9134885.0,7093910.0,54368620.0,2874593.0,...,2604157.0,7243920.0,1380660.0,1984970.0,7890626.0,1131020.0,486830.0,1124750.0,786399.0,0.0
2015-06-30,8595380.0,9310130.0,312970.0,1919645.0,810500.0,2769821.0,11888419.0,7412510.0,29620190.0,2393834.0,...,2497346.0,3445997.0,1306491.0,1225282.0,3695480.0,1264900.0,351120.0,1672421.0,1040340.0,27000.0
2015-09-30,7806580.0,13842600.0,421017.0,2127112.0,0.0,3518094.0,3319835.0,7813340.0,22095770.0,2173925.0,...,1945433.0,6278282.0,1929625.0,824650.0,3060123.0,805550.0,368907.0,1867501.0,1266187.0,1092360.0
2015-12-31,5295938.0,12001220.0,258101.0,1610145.0,2616260.0,3203889.0,14971650.0,10189630.0,19718490.0,1752725.0,...,2859402.0,3358701.0,2178481.0,2019050.0,3361951.0,1387190.0,2183420.0,1375997.0,1308847.0,1265040.0


In [ ]:
user_by_year.iloc[:,5]

구매일자
2014-12-31 00:00:00    1.139538e+07
2015-12-31 00:00:00    1.311104e+07
2014/2015 매출비율         8.691439e-01
Name: 6, dtype: float64

In [ ]:
# 2015년에 구매금액이 2014년 비교해서 감소한 고객 비율
user_by_q = user_df.resample('Q').sum()

In [ ]:
user_by_q.head()

고객번호,1,2,3,4,5,6,7,8,9,10,...,19374,19375,19376,19377,19378,19379,19380,19381,19382,19383
구매일자,,,,,,,,,,,,,,,,,,,,,
2014-03-31,9141590.0,14123470.0,302782.0,1309643.0,3145330.0,2166006.0,10635873.0,6431112.0,14363590.0,2545073.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-06-30,10783765.0,10199760.0,548433.0,3732463.0,2314820.0,3115284.0,2842322.0,5264270.0,23903830.0,2585853.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014-09-30,10781550.0,14841180.0,648188.0,2434410.0,1827290.0,3163273.0,4711569.0,6999500.0,16911080.0,2252934.0,...,0.0,0.0,0.0,0.0,0.0,130300.0,0.0,0.0,0.0,0.0
2014-12-31,23334762.0,7080920.0,394366.0,1823100.0,398440.0,2950816.0,16221723.0,11151680.0,22951170.0,1971990.0,...,817925.0,1159694.0,273297.0,217360.0,474760.0,1060320.0,522700.0,0.0,0.0,0.0
2015-03-31,9302985.0,18481480.0,279310.0,1737292.0,2124430.0,3619234.0,9134885.0,7093910.0,54368620.0,2874593.0,...,2604157.0,7243920.0,1380660.0,1984970.0,7890626.0,1131020.0,486830.0,1124750.0,786399.0,0.0


RFM


In [ ]:
# 구매금액이 큰 고객 분류
user_rev_sum = user_df.sum()
# user_rev_sum.sort_values(ascending = False, inplace = True)
user_rev_sum = pd.DataFrame(user_rev_sum)
# 구매횟수, 최근 구매일자 구하기
# user_freq_recency = df_purchase_tr[['고객번호','구매일자']].drop_duplicates()
# user_freq = pd.DataFrame(user_freq_recency.groupby('고객번호').구매일자.count())
# user_recency = pd.DataFrame(user_freq_recency.groupby('고객번호').구매일자.max())

# user_freq.rename(columns = {'구매일자':'Frequency'}, inplace = True)
# user_recency.rename(columns = {'구매일자':'Recency'}, inplace = True)
# user_rev_sum.rename(columns = {0:'Monetary'}, inplace = True)